In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

In [24]:
with open('./Data/list_actors.pickle', 'rb') as archivo:
    list_actors = pickle.load(archivo)

with open('./Data/list_directors.pickle', 'rb') as archivo:
    list_directors = pickle.load(archivo)

In [25]:
df_movie = pd.read_pickle('./Data/api.pickle')

In [9]:
df_movie.head()

,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,release_year,return,actors,directors,release_day,release_month
IdMovie,,,,,,,,,,,,,,
2,0.0,3.860491,1988-10-21,0.0,Released,Ariel,7.1,44.0,1988,0.000000,"[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",[Aki Kaurismäki],Friday,10
3,0.0,2.292110,1986-10-16,0.0,Released,Shadows in Paradise,7.1,35.0,1986,0.000000,"[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",[Aki Kaurismäki],Thursday,10
5,4000000.0,9.026586,1995-12-09,4300000.0,Released,Four Rooms,6.5,539.0,1995,1.075000,"[Tim Roth, Antonio Banderas, Jennifer Beals, M...","[Allison Anders, Alexandre Rockwell, Robert Ro...",Saturday,12
6,0.0,5.538671,1993-10-15,12136938.0,Released,Judgment Night,6.4,79.0,1993,0.000000,"[Emilio Estevez, Cuba Gooding Jr., Denis Leary...",[Stephen Hopkins],Friday,10
11,11000000.0,42.149697,1977-05-25,775398007.0,Released,Star Wars,8.1,6778.0,1977,70.490728,"[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",[George Lucas],Wednesday,5


In [29]:
df_movie.dtypes

budget                  float64
popularity              float64
release_date     datetime64[ns]
revenue                 float64
status                   object
title                    object
vote_average            float64
vote_count              float64
release_year              int64
return                  float64
actors                   object
directors                object
release_day              object
release_month             int64
dtype: object

In [30]:
df_movie.isna().sum()

budget              0
popularity          0
release_date        0
revenue             0
status             80
title               0
vote_average        0
vote_count          0
release_year        0
return              0
actors           2354
directors         723
release_day         0
release_month       0
dtype: int64

In [10]:
def cantidad_filmaciones_mes(mes:str):
    months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    if not isinstance(mes, str):
        return 'Debe ingresar un texto, por ejemplo, por ejemplo: enero'
    elif mes.lower() not in months:
        return 'Debe ingresar un mes válido del año, por ejemplo: enero'
    else:
        mes = mes.lower()
        month = months[mes]
        resp = df_movie[(df_movie['status'] == 'Released') & (df_movie['release_month'] == month)]['release_month'].count()
        return {'mes': mes, 'cantidad': str(resp)}

In [11]:
months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12,
        "sabado":0,
        12:0
    }
for a in months:
    print(cantidad_filmaciones_mes(a))

{'mes': 'enero', 'cantidad': '5831'}
{'mes': 'febrero', 'cantidad': '3006'}
{'mes': 'marzo', 'cantidad': '3532'}
{'mes': 'abril', 'cantidad': '3423'}
{'mes': 'mayo', 'cantidad': '3325'}
{'mes': 'junio', 'cantidad': '3131'}
{'mes': 'julio', 'cantidad': '2630'}
{'mes': 'agosto', 'cantidad': '3357'}
{'mes': 'septiembre', 'cantidad': '4798'}
{'mes': 'octubre', 'cantidad': '4581'}
{'mes': 'noviembre', 'cantidad': '3629'}
{'mes': 'diciembre', 'cantidad': '3766'}
Debe ingresar un mes válido del año, por ejemplo: enero
Debe ingresar un texto, por ejemplo, por ejemplo: enero


In [12]:
def cantidad_filmaciones_dia(dia:str):
    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday'}

    if not isinstance(dia, str):
        return 'Debe ingresar un texto, por ejemplo: sabado'
    elif dia.lower() not in days:
        return 'Debe ingresar un dia de la semana, por ejemplo: sabado'
    else:
        dia = dia.lower()
        day = days[dia]
        resp = df_movie[(df_movie['status'] == 'Released') & (df_movie['release_day'] == day)]['release_day'].count()
        return {'dia':dia, 'cantidad':resp}

In [13]:
days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday',
    "enero": 0,
    12:0}
for a in days:
    print(cantidad_filmaciones_dia(a))

{'dia': 'lunes', 'cantidad': 3474}
{'dia': 'martes', 'cantidad': 4598}
{'dia': 'miércoles', 'cantidad': 6986}
{'dia': 'jueves', 'cantidad': 7466}
{'dia': 'viernes', 'cantidad': 13783}
{'dia': 'sábado', 'cantidad': 5112}
{'dia': 'domingo', 'cantidad': 3590}
Debe ingresar un dia de la semana, por ejemplo: sabado
Debe ingresar un texto, por ejemplo: sabado


In [14]:
def score_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    else:
        resp = df_movie[df_movie['title'] == titulo][['title','popularity','release_year']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'popularidad':resp['popularity'], 'año':resp['release_year']}

In [15]:
print(score_titulo('123'))
print(score_titulo('jumanji'))
print(score_titulo(6534))

La pelicula no se encuentra en la base de datos
La pelicula no se encuentra en la base de datos
Debe ingresar un texto


In [16]:
def votos_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    elif int(df_movie[df_movie['title'] == titulo]['vote_count']) > 2000:
        resp = df_movie[df_movie['title'] == titulo][['title','vote_count','vote_average']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'voto_total':resp['vote_count'], 'voto_promedio':resp['vote_average']}
    else:
        return 'La consulta no cumple con las condicion por ende no se devuelve informacion'

In [17]:
votos_titulo('sgw54')

'La pelicula no se encuentra en la base de datos'

In [18]:
def get_actor(nombre_actor:str):
    if not isinstance(nombre_actor, str):
        return 'Debe ingresar un texto'
    elif nombre_actor not in list_actors:
        return 'El actor no se encuentra en la base de datos'

    _return = 0
    n_films = 0
    avg = 0
    for index,lista in enumerate(df_movie['actors']):
        if type(lista) == list: 
            if nombre_actor in lista:
                n_films += 1
                _return += df_movie['return'][index]
    
    if _return == 0 and n_films == 0:
        avg = 0
    else:
        avg = _return/n_films

    return {'actor':nombre_actor, 'cantidad_filmaciones':n_films, 'retorno_total':_return, 'retorno_promedio':avg}

In [21]:
get_actor('Tim Roth')

{'actor': 'Tim Roth',
 'cantidad_filmaciones': 60,
 'retorno_total': 59.652523166146054,
 'retorno_promedio': 0.9942087194357676}

In [24]:
df_movie['actor'].fillna('',inplace=True)

In [27]:
def get_director(nombre_director:str):
    if not isinstance(nombre_director, str):
        return 'Debe ingresar un texto'
    elif nombre_director not in list_directors:
        return 'El director no se encuentra en la base de datos'

    total_return = 0
    movies = []
    years = []
    _return = []
    budget = []
    revenue = []
    for index,lista in enumerate(df_movie['directors']):
        if type(lista) == list:
            if nombre_director in lista:
                total_return += df_movie['return'][index]
                movies.append(df_movie['title'][index])
                years.append(df_movie['release_year'][index])
                _return.append(df_movie['return'][index])
                budget.append(df_movie['budget'][index])
                revenue.append(df_movie['revenue'][index])

    return {'director':nombre_director, 'retorno_total_director':total_return, 
    'peliculas':movies, 'año':years, 'retorno_pelicula':_return, 
    'budget_pelicula':budget, 'revenue_pelicula':revenue}

In [148]:
list_directors = []
for row in df_movie['directors']:
    if type(row) == list:
        for director in row:
            if director not in list_directors:
                list_directors.append(director)

with open('../Data/list_directors.txt', 'w') as file:
    json.dump(list_directors, file)

In [29]:
get_director('Aki Kaurismäki')

{'director': 'Aki Kaurismäki',
 'retorno_total_director': 1.1703028985507247,
 'peliculas': ['Ariel',
  'Shadows in Paradise',
  'Lights in the Dusk',
  'The Man Without a Past',
  'The Match Factory Girl',
  'I Hired a Contract Killer',
  'Drifting Clouds',
  'Hamlet Goes Business',
  'Visions of Europe',
  'Leningrad Cowboys Go America',
  'Ten Minutes Older: The Trumpet',
  'La Vie de Bohème',
  'Leningrad Cowboys Meet Moses',
  'To Each His Own Cinema',
  'Juha',
  'Take Care of Your Scarf, Tatiana',
  'Calamari Union',
  'Rocky VI',
  'Le Havre',
  'Crime and Punishment',
  'The Saimaa Gesture',
  'Dirty Hands',
  'Thru the Wire',
  'The Other Side of Hope'],
 'año': [1988,
  1986,
  2006,
  2002,
  1990,
  1990,
  1996,
  1987,
  2004,
  1989,
  2002,
  1992,
  1994,
  2007,
  1999,
  1994,
  1985,
  1986,
  2011,
  1983,
  1981,
  1989,
  1987,
  2017],
 'retorno_pelicula': [0.0,
  0.0,
  1.1703028985507247,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
 